Download overlay and grab DMA aliases

<div class="alert alert-box alert-warning">
For <b>Kria</b> boards use the `Debug Bridge` IP instead of the JTAG cable.
</div>

In [ ]:
from pynq import Overlay, allocate, PL
import numpy as np

PL.reset()

ol = Overlay('dma_notkeep.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In Vivado open **Open Hardware Manger**, then **Open Target** then **Auto Connect**

Allocate input and output buffer, then initialize input buffer

In [ ]:
data_size = 20

input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

input_buffer[:] = np.arange(data_size, dtype=np.uint32)

In Vivado Hardware Manager, set the trigger to start capturing when `TVALID` of *axis_data_fifo_M_AXIS* is 1 or in the rising edge.

Set the number of windows to 1 with a depth of 1024 and the trigger position to 0

This will lead to `dma_recv.wait()` not to complete, after a few second use the `Interrupt Kernel button`

In [ ]:
dma_recv.transfer(output_buffer)
dma_send.transfer(input_buffer)
dma_send.wait()

In [ ]:
dma_recv.wait()

Unfortunately, the `dma.register_map` does not provide useful information.

In [ ]:
dma.register_map

Release the buffers and overlay

In [ ]:
del input_buffer, output_buffer
ol.free()